### 1. Check the Discriminator Model

In [1]:
import torchvision.models as models
import torch.nn as nn

In [2]:
class ResNet18(object):
    """ResNet model with modified output layer"""
    def __init__(self, input_channel=1, output_dim=2):
        super().__init__()
        # extract fc layers features
        self.model = models.resnet18()
        num_features = self.model.fc.in_features     
        self.model.fc = nn.Linear(num_features, output_dim)
        if input_channel !=3:
            self.model.conv1 = nn.Conv2d(input_channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) 

    def get_model(self):
        return self.model

In [3]:
discriminator = ResNet18().get_model()

In [4]:
import torch

In [5]:
imput_tensor = torch.Tensor(5, 1, 512, 512)

In [6]:
output = discriminator(imput_tensor)

In [7]:
output

tensor([[ 0.0225, -0.0084],
        [ 0.0225, -0.0084],
        [ 0.0225, -0.0084],
        [ 0.0225, -0.0084],
        [ 0.0225, -0.0084]], grad_fn=<AddmmBackward0>)

In [8]:
output.shape

torch.Size([5, 2])

### 2. Check the Generator Model

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [10]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, depth=4):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.depth = depth

        init_n = 64
        self.inc = DoubleConv(n_channels, init_n)

        for iDep in range(depth):
            setattr(self, f'down{iDep+1}', 
                Down(init_n*2**iDep, init_n*2**(iDep+1)))
            # e.g. when the depth is 4, the following 'down' layers would be creates 
                # self.down1 = Down(64, 128)
                # self.down2 = Down(128, 256)
                # self.down3 = Down(256, 512)
                # self.down4 = Down(512, 1024)
            setattr(self, f'up{iDep+1}', 
                Up(init_n*2**(depth-iDep), init_n*2**(depth-iDep-1)))
            # e.g. when the depth is 4, the following 'up' layers would be creates 
                # self.up1 = Up(1024, 512)
                # self.up2 = Up(512, 256)
                # self.up3 = Up(256, 128)
                # self.up4 = Up(128, 64)

        self.outc = OutConv(init_n, n_classes)

    def forward(self, x):
        depth = self.depth
        setattr(self, 'x0', self.inc(x)) # self.x0 = self.inc(x)
        for iDep in range(depth):
            setattr(self, f'x{iDep+1}', 
                getattr(self, f'down{iDep+1}')(getattr(self, f'x{iDep}'))
                )
        # e.g. when the depth is 4, forward through the following 'down' layers  
            # x1 = self.down1(x0)
            # x2 = self.down2(x1)
            # x3 = self.down3(x2)
            # x4 = self.down4(x3)

        for iDep in range(depth):
            setattr(self, f'x{depth-iDep}', 
                getattr(self, f'up{iDep+1}')(
                    getattr(self, f'x{depth-iDep}'), 
                    getattr(self, f'x{depth-iDep-1}')
                    )
                )
        # e.g. when the depth is 4, forward through the following 'up' layers    
            # x3 = self.up1(x4, x3)
            # x2 = self.up2(x3, x2)
            # x1 = self.up3(x2, x1)
            # x0 = self.up4(x1, x0)

        logits = self.outc(self.x0)
        
        return logits

In [11]:
generator = UNet(2, 1, 2)

In [12]:
imput_tensor = torch.Tensor(5, 2, 512, 512)

In [13]:
output = generator(imput_tensor)

In [14]:
output.shape

torch.Size([5, 1, 512, 512])

In [16]:
output[1, :, :, :]

tensor([[[-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659],
         [-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659],
         [-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659],
         ...,
         [-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659],
         [-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659],
         [-0.0659, -0.0659, -0.0659,  ..., -0.0659, -0.0659, -0.0659]]],
       grad_fn=<SliceBackward0>)

### 3. Parameter Number

In [59]:
def count_parameters(model):
    total_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The total number of parameter is {total_param/1e6 :.2f}')

In [60]:
count_parameters(generator)

The total number of parameter is 1.86


In [55]:
count_parameters(resnet18)

11171266